In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import html
import re
from fastai.text import *
from tqdm import tqdm
from sklearn import model_selection

C:\Users\PC\Anaconda3\envs\tflow\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [34]:
LANG = 'tagalog'
LM_PATH = Path(f'{LANG}/')
LM_PATH_MODEL = f'models/wiki_tagalog_lm4.h5'
LM_PATH_ITOS = f'wiki_{LANG}_itos.pkl'
PATH_TMP = Path('tmp/')

In [35]:
itos = pickle.load(open(LM_PATH_ITOS, 'rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
vs = len(itos)
print("vs:", vs)

vs: 43006


In [36]:
from functools import partial
import torch.optim as optim

em_sz = 400  # size of each embedding vector
nh = 1150    # number of hidden activations per layer
nl = 3       # number of layers

wd = 1e-7
bptt = 70
bs = 64
# opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
opt_fn = partial(optim.SGD, momentum=0.9)
weight_factor = 0.3
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*weight_factor


# Loading the indexed representation of our dataset from disk
# we also load the index-word mapping to to help us convert the indexes to word datasets, if need be.
trn_lm = np.load(PATH_TMP/'wiki_trn_ids.npy')
val_lm = np.load(PATH_TMP/'wiki_val_ids.npy')
wiki_itos = pickle.load(open(f'wiki_{LANG}_itos.pkl', 'rb'))

trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
md = LanguageModelData('.', 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.load(f'wiki_{LANG}_lm4')
learner.load_encoder(f'wiki_{LANG}_lm_enc4')

m = learner.model # initialized model
m.eval() # ensure you switch your model to inference mode
m[0].bs = 1 # set batch size to 1

In [37]:
def gen_text(ss, topk):
    s = ss.strip().split(" ")
    t = LongTensor([stoi[i] for i in s]).view(-1, 1).cuda()
    t = Variable(t, volatile=False)
    m.reset()
    pred, *_ = m(t)
    pred_i = torch.topk(pred[-1], topk)[1]

    return [itos[o] for o in to_np(pred_i)]

def generate_sentences(ss, nb_words):
    result = []
    s = ss.strip().split(" ")
    t = LongTensor([stoi[i] for i in s]).view(-1, 1).cuda()
    t = Variable(t, volatile=False)
    m.reset()
    pred, *_ = m(t)
    
    for i in range(nb_words):
        pred_i = pred[-1].topk(2)[1]
        pred_i = pred_i[1] if pred_i.data[0] < 2 else pred_i[0]
        word = itos[pred_i.data[0]]
        if word != "xbos":
            result.append(word)
        else:
            break
        pred, *_ = m(pred_i.unsqueeze(0).unsqueeze(0))
    
    result = re.sub('\s+([.,])', r'\1', "{} {}".format(ss, " ".join(result).rstrip()))

    return(result)

In [38]:
strings = ["Pilipinas", 
           "Lalaki", 
           "Babae",
           "Diyos", 
           "Pampanga",
           "Agham",
           "Masaya",
           "Aso",
           "Rizal",
           "Quezon",
           "Kanina",
           "Manila"
           ]

In [39]:
import warnings
warnings.filterwarnings('ignore')
for s in strings:
    print(f"{s}... \n{generate_sentences(s.lower(), 80)}\n")

Pilipinas... 
pilipinas. 
 ang mga sumusunod ay ang mga dahilan ng kanilang mga pag - atake. ayon sa mga tala, ang mga pilipino ay dapat na magkaroon ng mga anak na lalaki sa isang paaralan o kolehiyo. ang mga ito ay mga opisyal na panuluyan ng mga pari, mga pari at mga pari. ang mga pari ay maaaring mag - alok ng mga pondo para sa kanilang mga layunin at mga gawain. ang mga mag

Lalaki... 
lalaki at babae. ang mga babae ay may mga pagkakataon na ang mga babae ay may mas mataas na antas ng sekswal na orientasyon kaysa sa mga babae. ang mga babae ay may mas mataas na antas ng pag - iisip kaysa sa mga babae. ang mga babae ay may mataas na antas ng pagiging lalaki at babae, at ang mga babae ay may mataas na pagiging pambabae, at ang mga babae ay may mas mataas

Babae... 
babae at babae. ang mga babae ay may mga pagkakataon na ang mga babae ay may mas mataas na antas ng sekswal na orientasyon kaysa sa mga babae. ang mga babae ay may mas mataas na antas ng pag - iisip kaysa sa mga babae. an